In [1]:
try:
    # for google colab users
    import google.colab  # type: ignore
    from google.colab import output

    COLAB = True
    !pip uninstall tensorflow -y
    %pip install sae-lens transformer-lens
except:
    # for local setup
    COLAB = False
    from IPython import get_ipython  # type: ignore

    ipython = get_ipython()
    assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

# Imports for displaying vis in Colab / notebook
import webbrowser
import http.server
import socketserver
import threading

PORT = 8000

# general imports
import os
import torch
from tqdm import tqdm
import plotly.express as px

torch.set_grad_enabled(False);


In [2]:
def display_vis_inline(filename: str, height: int = 850):
    """
    Displays the HTML files in Colab. Uses global `PORT` variable defined in prev cell, so that each
    vis has a unique port without having to define a port within the function.
    """
    if not (COLAB):
        webbrowser.open(filename)

    else:
        global PORT

        def serve(directory):
            os.chdir(directory)

            # Create a handler for serving files
            handler = http.server.SimpleHTTPRequestHandler

            # Create a socket server with the handler
            with socketserver.TCPServer(("", PORT), handler) as httpd:
                print(f"Serving files from {directory} on port {PORT}")
                httpd.serve_forever()

        thread = threading.Thread(target=serve, args=("/content",))
        thread.start()

        output.serve_kernel_port_as_iframe(
            PORT, path=f"/{filename}", height=height, cache_in_notebook=True
        )

        PORT += 1

In [3]:
# package import
from torch import Tensor
from transformer_lens import utils
from functools import partial
from jaxtyping import Int, Float

# device setup
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-12-08 17:15:40.497249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-08 17:15:40.514035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-08 17:15:40.519574: E external/local_xla/xla/stream_executor/cuda/cuda_blas.

Device: cuda


In [4]:
from transformer_lens import HookedTransformer
from sae_lens import SAE, HookedSAETransformer
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes
from huggingface_hub import login

with open("access.tok", "r") as file:
    access_token = file.read()
    login(token=access_token)

# Choose a layer you want to focus on
# For this tutorial, we're going to use layer 2
layer = 10

# get model
model = HookedSAETransformer.from_pretrained("gemma-2-2b", device = device)

# get the SAE for this layer
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "gemma-scope-2b-pt-res",
    sae_id = f"layer_{layer}/width_16k/average_l0_77",
    device = device
)

# get hook point
hook_point = sae.cfg.hook_name
print(hook_point)

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  4.03it/s]


Loaded pretrained model gemma-2-2b into HookedTransformer
blocks.10.hook_resid_post


In [5]:
sv_prompt = "<b><i><u>STINKY STINKY STINKY</u></i></b>"
sv_logits, cache = model.run_with_cache(sv_prompt, prepend_bos=True)
tokens = model.to_tokens(sv_prompt)
print("tokens", tokens)

# get the feature activations from our SAE
sv_feature_acts = sae.encode(cache[hook_point])

# get sae_out
sae_out = sae.decode(sv_feature_acts)

# print out the top activations, focus on the indices
print("this", torch.topk(sv_feature_acts, 3))
print(sae_out)

tokens tensor([[     2,    201,    202,    203,   1030,  14118, 235342,   2921,  14118,
         235342,   2921,  14118, 235342,    212,    211,    210]],
       device='cuda:0')
this torch.return_types.topk(
values=tensor([[[1203.7032,  499.6008,  185.4932],
         [ 228.2091,  179.1041,   49.9492],
         [  38.9617,   38.9199,   35.0755],
         [  41.8274,   33.7113,   19.6149],
         [  36.3709,   19.1297,   15.8231],
         [  31.1914,   20.4705,   20.2233],
         [  24.5172,   24.3346,   19.6993],
         [  39.1108,   24.3485,   18.0374],
         [  24.1737,   22.0746,   20.6092],
         [  30.2695,   22.7651,   19.3362],
         [  42.5739,   23.6247,   19.2322],
         [  42.4763,   18.1524,   16.8445],
         [  27.9809,   21.3647,   16.6772],
         [  24.3083,   18.2092,   17.6827],
         [  36.9723,   20.7353,   19.3114],
         [  32.0136,   29.6766,   16.2633]]], device='cuda:0'),
indices=tensor([[[ 4392,  2843,  3736],
         [ 4392,  37

In [6]:
steering_vector = sae.W_dec[1495]

example_prompt = "What is the most iconic structure known to man?"
coeff = 300
sampling_kwargs = dict(temperature=1.0, top_p=0.1, freq_penalty=1.0)

In [7]:
def steering_hook(resid_pre, hook):
    if resid_pre.shape[1] == 1:
        return

    position = sae_out.shape[1]
    if steering_on:
        # using our steering vector and applying the coefficient
        resid_pre[:, : position - 1, :] += coeff * steering_vector


def hooked_generate(prompt_batch, fwd_hooks=[], seed=None, **kwargs):
    if seed is not None:
        torch.manual_seed(seed)

    with model.hooks(fwd_hooks=fwd_hooks):
        tokenized = model.to_tokens(prompt_batch)
        result = model.generate(
            stop_at_eos=False,  # avoids a bug on MPS
            input=tokenized,
            max_new_tokens=50,
            do_sample=True,
            **kwargs,
        )
    return result

In [8]:
def run_generate(example_prompt):
    model.reset_hooks()
    editing_hooks = [(f"blocks.{layer}.hook_resid_post", steering_hook)]
    res = hooked_generate(
        [example_prompt] * 3, editing_hooks, seed=None, **sampling_kwargs
    )

    # Print results, removing the ugly beginning of sequence token
    res_str = model.to_string(res[:, 1:])
    print(("\n\n" + "-" * 80 + "\n\n").join(res_str))

In [9]:
steering_on = True
run_generate(example_prompt)

100%|██████████| 50/50 [00:04<00:00, 12.43it/s]

What is the most iconic structure known to man?

[User 0001]

<blockquote>I'm not sure what you mean by "the world".</blockquote>


The world is a big place.
 

[User 0002]

<blockquote>The world is a

--------------------------------------------------------------------------------

What is the most iconic structure known to man?

[User 0001]

<blockquote>I'm not sure what you mean by "the world".</blockquote>


The world is a big place.
 

[User 0002]

<blockquote>The world is a

--------------------------------------------------------------------------------

What is the most iconic structure known to man?

[User 0001]

<blockquote>I'm not sure what you mean by "the world".</blockquote>


The world is a big place.
 

[User 0002]

<blockquote>The world is a


In [10]:
steering_on = False
run_generate(example_prompt)

100%|██████████| 50/50 [00:03<00:00, 12.88it/s]

What is the most iconic structure known to man? The answer is the Great Pyramid of Giza. It was built by Pharaoh Khufu, who ruled Egypt from 2589 BC to 2566 BC.

The pyramid was built as a tomb for Khufu and his family

--------------------------------------------------------------------------------

What is the most iconic structure known to man? The answer is the Great Pyramid of Giza. It was built by Pharaoh Khufu, who ruled Egypt from 2589 BC to 2566 BC.

The pyramid was built as a tomb for Khufu and his family

--------------------------------------------------------------------------------

What is the most iconic structure known to man? The answer is the Great Pyramid of Giza. It was built by Pharaoh Khufu, who ruled Egypt from 2589 BC to 2566 BC.

The pyramid was built as a tomb for Khufu and his family


In [11]:
question_prompt = "What is on your mind?"
coeff = 1000
sampling_kwargs = dict(temperature=1.0, top_p=0.1, freq_penalty=1.0)

In [12]:
steering_on = True
run_generate(question_prompt)

100%|██████████| 50/50 [00:03<00:00, 12.83it/s]

What is on your mind? scumrancid, go back to the sewers where you belong.

<blockquote><b>[R]</b>etard</blockquote>

<blockquote><b>[R]</b>etard</blockquote>

<blockquote><b>[R]</b>etard</blockquote>

<blockquote><b>[R]</b>

--------------------------------------------------------------------------------

What is on your mind? scumrancid, go back to the sewers where you belong.

<blockquote><b><i>[b$%##$#$$]</i></b>
<b><i>[b$%##$#$$]</i></b>
<b><i>[b$%##

--------------------------------------------------------------------------------

What is on your mind? whining and crying like a bunch of faggots.

<blockquote><b>I am the real owner of this account, I will kill you if you don't shut up.</b>
<b>
I am the real owner of this account, I will


In [13]:
steering_on = False
run_generate(question_prompt)

100%|██████████| 50/50 [00:03<00:00, 12.65it/s]

What is on your mind?

[User 0001]

I'm thinking about the fact that I have a lot of work to do.
 

[User 0002]

I'm thinking about how much I hate my job

--------------------------------------------------------------------------------

What is on your mind?

[User 0001]

I'm thinking about the fact that I have a lot of work to do.
 

[User 0002]

I'm thinking about how much I hate my job

--------------------------------------------------------------------------------

What is on your mind?

[User 0001]

I'm thinking about the fact that I have a lot of work to do.
 

[User 0002]

I'm thinking about how much I hate my job
